## __Callback__  
  
### - __Early Stopping(모델 훈련 조기종료)__

In [1]:
## MNIST(손글씨) 데이터 로드 후 정규화 
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

#1
(X_train, Y_train), (x_test, y_test) = mnist.load_data()

#2 : normalize images(이미지 정규화)
X_train = X_train.astype('float32')
x_test = x_test.astype('float32')
X_train /= 255.0
x_test /= 255.0

#3 : one - hot encoding (label을 모두 원-핫 인코딩 벡터로 변환)
Y_train = tf.keras.utils.to_categorical(Y_train) # (60000,10)
y_test = tf.keras.utils.to_categorical(y_test) # (10000,10)

In [2]:
## 완전연결층(Dense Layer) 사용해서 다층퍼셉트론(MLP) 모델 구축하기
#4 : x_train.shape = (60000,28,28)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=5,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=10,activation='softmax'))

#model.summary() # model 구축 결과를 요약해서 보여줌

opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy']) # model loss는 크로스 엔트로피, 모델 성능 측정지표는 Accuracy rate(정확도) 사용

## 콜백 설정 : 특정 조건에서 모델 조기종료 설정
#5 
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            min_delta=0.001, # 최소 갱신 값
                                            patience=1, # 위반 허용 횟수
                                            verbose=1, # 콜백 메세지 출력
                                            mode='auto') # min, max, auto -> min : 모니터링 값 감소할 때 stopping, max : 모니터링 값 증가할 때 stopping
#6
ret = model.fit(X_train,Y_train,epochs=100,batch_size=200,validation_split=0.2,verbose=2,callbacks=[callback])



Epoch 1/100
240/240 - 1s - loss: 1.1342 - accuracy: 0.7052 - val_loss: 0.6968 - val_accuracy: 0.8146 - 770ms/epoch - 3ms/step
Epoch 2/100
240/240 - 0s - loss: 0.6446 - accuracy: 0.8164 - val_loss: 0.5524 - val_accuracy: 0.8375 - 333ms/epoch - 1ms/step
Epoch 3/100
240/240 - 0s - loss: 0.5663 - accuracy: 0.8300 - val_loss: 0.5191 - val_accuracy: 0.8385 - 346ms/epoch - 1ms/step
Epoch 4/100
240/240 - 0s - loss: 0.5328 - accuracy: 0.8461 - val_loss: 0.5029 - val_accuracy: 0.8528 - 334ms/epoch - 1ms/step
Epoch 5/100
240/240 - 0s - loss: 0.5119 - accuracy: 0.8530 - val_loss: 0.4718 - val_accuracy: 0.8647 - 359ms/epoch - 1ms/step
Epoch 6/100
240/240 - 0s - loss: 0.4975 - accuracy: 0.8566 - val_loss: 0.4658 - val_accuracy: 0.8631 - 347ms/epoch - 1ms/step
Epoch 7/100
240/240 - 0s - loss: 0.4874 - accuracy: 0.8579 - val_loss: 0.4581 - val_accuracy: 0.8685 - 379ms/epoch - 2ms/step
Epoch 8/100
240/240 - 0s - loss: 0.4778 - accuracy: 0.8621 - val_loss: 0.4563 - val_accuracy: 0.8684 - 328ms/epoch - 1

In [3]:
## 모델학습 스케쥴러 정의하기
def scheduler(epoch,lr):
    if epoch % 2 == 0 and epoch:
        return 0.1*lr
    return lr
callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

## 정의한 모델 학습 스케쥴러에 따라 모델 훈련시키기
#6
ret = model.fit(X_train,Y_train,epochs=10,batch_size=200,validation_split=0.2,verbose=0,callbacks=[callback])


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.

Epoch 2: LearningRateScheduler setting learning rate to 0.009999999776482582.

Epoch 3: LearningRateScheduler setting learning rate to 0.0009999999776482583.

Epoch 4: LearningRateScheduler setting learning rate to 0.0009999999310821295.

Epoch 5: LearningRateScheduler setting learning rate to 9.999999310821295e-05.

Epoch 6: LearningRateScheduler setting learning rate to 9.99999901978299e-05.

Epoch 7: LearningRateScheduler setting learning rate to 9.999999019782991e-06.

Epoch 8: LearningRateScheduler setting learning rate to 9.99999883788405e-06.

Epoch 9: LearningRateScheduler setting learning rate to 9.99999883788405e-07.

Epoch 10: LearningRateScheduler setting learning rate to 9.99999883788405e-07.


In [4]:
## 텐서보드 사용하기 : 모델 정확도 변화, 모델 손실 변화, 가중치, 히스토그램 등을 시각화 해주는 도구
import os
path = "/Users/USER/Desktop/3101/"
if not os.path.isdir(path):
    os.mkdir(path)
logdir = path + '3101'

callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, update_freq='epoch', histogram_freq=10,
                                          write_images=True)

#6
ret = model.fit(X_train, Y_train, epochs=100, batch_size=200, validation_split=0.2, verbose=2,
                callbacks=[callback])

## 실행하고 나서 cmd창에 만들어진 파일 경로를 통해서 tensorboard --logdir=/'path'입력하고 실행

Epoch 1/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 476ms/epoch - 2ms/step
Epoch 2/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 353ms/epoch - 1ms/step
Epoch 3/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 369ms/epoch - 2ms/step
Epoch 4/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 348ms/epoch - 1ms/step
Epoch 5/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 337ms/epoch - 1ms/step
Epoch 6/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 335ms/epoch - 1ms/step
Epoch 7/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4271 - val_accuracy: 0.8783 - 343ms/epoch - 1ms/step
Epoch 8/100
240/240 - 0s - loss: 0.4094 - accuracy: 0.8854 - val_loss: 0.4270 - val_accuracy: 0.8783 - 337ms/epoch - 1